# Building a Conversational Chatbot for Slack using Rasa and Python -Part 1

## Starting Jupyter Notebook with necessary imports

In [11]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')



# Installations
* Rasa NLU
* Rasa Core
* SpaCy Language Model

In [12]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -U rasa_core==0.9.6 rasa_nlu[spacy];

  Using cached https://files.pythonhosted.org/packages/72/de/82e762685a991351a9f89d934b1b25fe04d10462cf4017f93227cb4d9058/rasa_core-0.9.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d6/53/31b6b14e124fa916c10e8e58d650cff57ace0027a9e69a0788b1afcc26f4/rasa_nlu-0.14.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f1/51/d1bd383522c12b313eddd7b97b8e7d6cd2a8e3b44b8ff3c88e4a7b045cc8/python_telegram_bot-10.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/03/ad/d732a5d9d50bfcd8aeb6e4a266065a8868829388e4e2b529ff689f1fc923/tensorflow-1.8.0-cp36-cp36m-macosx_10_11_x86_64.whl
^C
Operation cancelled by user


In [13]:
!{python} -m spacy download en_core_web_md


    Linking successful
    /usr/local/lib/python3.6/site-packages/en_core_web_md -->
    /usr/local/lib/python3.6/site-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



## Downloading the English Language Model

In [14]:
!{python} -m spacy link en_core_web_md en --force;


    Linking successful
    /usr/local/lib/python3.6/site-packages/en_core_web_md -->
    /usr/local/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [15]:
!pip install rasa_core

# Importing the Installations

In [31]:
import rasa_nlu
import rasa_core
import spacy



# 1. Teaching the bot to understand user inputs using Rasa NLU

## Preparing the NLU Training Data

nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really




## intent:student_work_authorization
- What does the student’s work authorization look like?
- how is student work authorization?
- how does student work authorization look like?
- Student work authorization look?
- what's Student work authorization

## intent:student_work
- When can a student work?
- When can I work?


## intent:student_work_timings
- How many hours per week can a student work?
- How many hours can I work?
- number of hours I can work?
- no. of hours I can work?

## intent:taxes
- Are F-1 students subject to taxes?
- am I subjected to tax ?
- how much tax shoud I pay ?
- should I pay tax ?

## intent:E_verify
- What is E-Verify?
- how to do e-verify?
- What is E Verify?
- how to do e verify?
- What is EVerify?
- how to do everify

##intent: Visa_interview
-What all do I need for Visa Interview?
-what all documents do I need for Visa Interview?
-Documents needed for Visa Interview?
-What all things do I need for Visa Interview?
-What all papaers do I need for Visa Interview?

##intent: travel_date
-When can I travel to the usa?
-How many days prior to my admission can I travel to the USA?

## intent:customs_immigration
-What all do I need for customs and immigration?
-what all documents do I need for customs and immigration?
-Documents needed for customs and immigration?
-What all things do I need for customs and immigration?
-What all papaers do I need for customs and immigration?

##intent:questions_during_visa
-What are the common questions asked during Visa interview?
-What should I prepare for Visa interview?
-What kind of questions are asked during a visa interview?
-Common questions durin Visa interview?
-Questions asked in a Visa interview?

##intent:health_insurance
-How do I pyrchase University heath insurance?
-What steps are involved in purchasing student health insurance?
-Steps to purchase health insurance?
-international Heath Insurance puchase?

##intent:immunization
-What are the steps involved in immunization?
-What all immunization do I need before getting to the USA?
-Kind of immunization required before reaching?
-What is the proof of immunization that needs to be submitted?

##intent:housing
-Where can I find the housing details of SJSU?
-Accomodation in SJSU?
-Accomodation for international students at SJSU?
-Housing for students near SJSU?
-Housing on campus?

##intent:course_registration
-How many courses do I need to register for?
-How many credits are mandatory?
-Number of credits at SJSU?
-SJSU credits for F1 students?

##intent:tution
-Where can I find details about my tution fees?
-Information about tution fees?
-When do I have to pay for tution fees?
-When can I register for the courses?
-Where do I pay my tution?

##intent:tower_ID
-Where can I get my ID?
-Do I need an ID?
-Procedure to get an ID?
-Documents required to get an ID?
-How do I get my student ID?
-Steps involved in getting student ID card?

## intent: inform
- A [dog](group:shibes)
- [dog](group:shibes)
- [bird](group:birds)
- a [cat](group:cats)
- [cat](group:cats)
- a [bird](group:birds)
- of a [dog](group:shibes)
- of a [cat](group:cats)
- a [bird](group:birds), please
- a [dog](group:shibes), please
"""

%store nlu_md > nlu.md

In [17]:
%ls

chatbot_config.py                nlu_model.py
config.yml                       policy_config.yml
domain.yml                       slack_connector.py
errors.json                      stories.md
models/                          stories_backup.md
nlu.md                           story_graph.png
nlu_config.yml                   student_assistant_chatbot.ipynb


## Defining the NLU Model Configuration

In [67]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"
- name: "tokenizer_spacy"
- name: "intent_entity_featurizer_regex"
- name: "intent_featurizer_spacy"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_classifier_sklearn"
"""

%store config > nlu_config.yml

Writing 'config' (str) to file 'nlu_config.yml'.


In [68]:
config = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"
- name: "ner_duckling_http"
  url: "http://127.0.0.1:8000"
  dimensions: ["time"] 
"""

%store config > nlu_config.yml

Writing 'config' (str) to file 'nlu_config.yml'.


## Training the NLU Model.

In [69]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 104 (20 distinct intents)
	- Found intents: 'student_work_timings', 'tution', 'questions_during_visa', 'E_verify', 'health_insurance', 'housing', 'student_work', 'greet', 'tower_ID', 'goodbye', 'travel_date', 'inform', 'student_work_authorization', 'course_registration', 'customs_immigration', 'taxes', 'immunization', 'mood_affirm', 'Visa_interview', 'mood_deny'
	- entity examples: 3 (1 distinct entities)
	- found entities: 'events'

INFO:rasa_nlu.model:Starting to train component tokenizer_whitespace
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_crf
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_synonyms
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_count_

## Evaluating the NLU model on a random text

In [70]:
# A helper function for prettier output
import json
def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("is there any event on day after tomorrow ? "))

{
  "intent": {
    "name": "inform",
    "confidence": 0.9217749834060669
  },
  "entities": [
    {
      "start": 13,
      "end": 18,
      "value": "event",
      "entity": "events",
      "confidence": 0.8924925786107403,
      "extractor": "ner_crf"
    },
    {
      "start": 19,
      "end": 40,
      "text": "on day after tomorrow",
      "value": "2019-04-18T00:00:00.000-07:00",
      "confidence": 1.0,
      "additional_info": {
        "values": [
          {
            "value": "2019-04-18T00:00:00.000-07:00",
            "grain": "day",
            "type": "value"
          }
        ],
        "value": "2019-04-18T00:00:00.000-07:00",
        "grain": "day",
        "type": "value"
      },
      "entity": "time",
      "extractor": "ner_duckling_http"
    }
  ],
  "intent_ranking": [
    {
      "name": "inform",
      "confidence": 0.9217749834060669
    },
    {
      "name": "goodbye",
      "confidence": 0.23540902137756348
    },
    {
      "name": "student_work

## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [71]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("nlu.md", model_directory)

INFO:tensorflow:Restoring parameters from /Users/saching12/Desktop/python codes/student_assistant_chatbot/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt


INFO:tensorflow:Restoring parameters from /Users/saching12/Desktop/python codes/student_assistant_chatbot/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt
INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 104 (20 distinct intents)
	- Found intents: 'student_work_timings', 'tution', 'questions_during_visa', 'E_verify', 'health_insurance', 'housing', 'student_work', 'greet', 'tower_ID', 'goodbye', 'travel_date', 'inform', 'student_work_authorization', 'course_registration', 'customs_immigration', 'taxes', 'immunization', 'mood_affirm', 'Visa_interview', 'mood_deny'
	- entity examples: 3 (1 distinct entities)
	- found entities: 'events'

INFO:rasa_nlu.evaluate:Skipping evaluation of ner_duckling_http
INFO:rasa_nlu.evaluate:Intent evaluation results:
INFO:rasa_nlu.evaluate:Intent Evaluation: Only considering those 104 examples that have a defined intent out of 

{'intent_evaluation': {'predictions': [{'text': 'hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9697084426879883},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9683195352554321},
   {'text': 'hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9640589952468872},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9683195352554321},
   {'text': 'good morning',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9776124358177185},
   {'text': 'good evening',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9719564318656921},
   {'text': 'moin',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9577948451042175},
   {'text': 'hey there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9623345732688904},
   {'text': "let's go",
    'intent': 'greet',
    'predicted': 'gre

# 2. Teaching the bot to respond using Rasa Core

## Custom Actions

In [80]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.core.display import Image, display

import requests

class action_retrieve_event(Action):
    def name(self):
        return "action_retrieve_event"

    def run(self, dispatcher, tracker, domain):
        
        time = tracker.get_slot('time')
        #date = time.split('T')[0]
        #print(date)
        #r = requests.get('http://shibe.online/api/{}?count=1&urls=true&httpsUrls=true'.format(time))
        #response = r.content.decode()
        #response = response.replace('["',"")
        #response = response.replace('"]',"")
   
        
        #display(Image(response[0], height=550, width=520))
        dispatcher.utter_message("Here is something to cheer you up: ")

##  Visualising the Training Data

## Install these packages
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
!breq install graphviz

!{python} -m pip install pygraphviz;

In [81]:

from IPython.display import Image
from rasa_core.agent import Agent

agent = Agent('domain.yml')
agent.visualize("stories.md", "story_graph.png", max_history=2)
#Image(filename="story_graph.png")

Processed Story Blocks: 100%|██████████| 49/49 [00:00<00:00, 1097.10it/s, # trackers=1]


In [82]:

config = """
policies:
  - name: "KerasPolicy"
    epochs: 700
    featurizer:
    - name: MaxHistoryTrackerFeaturizer
      max_history: 2
      state_featurizer:
        - name: BinarySingleStateFeaturizer
  - name: "MemoizationPolicy"
    max_history: 2
  - name: "FallbackPolicy"
    nlu_threshold: 0.5
    core_threshold: 0.2
    fallback_action_name: "utter_unclear"
""" 

%store config > policy_config.yml


Writing 'config' (str) to file 'policy_config.yml'.


## Training a Dialogue Model

In [83]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent
from rasa_core import config


agent = Agent('domain.yml', policies=config.load('policy_config.yml'))

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(training_data)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 49/49 [00:00<00:00, 509.68it/s, # trackers=4]
INFO:rasa_core.policies.keras_policy:Fitting model with 222 total samples and a validation split of 0.1


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 2, 56)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                11392     
_________________________________________________________________
dense (Dense)                (None, 30)                990       
_________________________________________________________________
activation (Activation)      (None, 30)                0         
Total params: 12,382
Trainable params: 12,382
Non-trainable params: 0
_________________________________________________________________
Epoch 1/700
222/222 [==============================] - 1s 3ms/step - loss: 3.3974 - acc: 0.0495
Epoch 2/700
222/222 [==============================] - 0s 106us/step - loss: 3.3727 - acc: 0.0811
Epoch 3/700
222/222 [==============================] - 0s 114us/step - loss:

222/222 [==============================] - 0s 113us/step - loss: 0.9771 - acc: 0.8243
Epoch 76/700
222/222 [==============================] - 0s 122us/step - loss: 0.9440 - acc: 0.8333
Epoch 77/700
222/222 [==============================] - 0s 106us/step - loss: 0.9294 - acc: 0.8378
Epoch 78/700
222/222 [==============================] - 0s 108us/step - loss: 0.8775 - acc: 0.8784
Epoch 79/700
222/222 [==============================] - 0s 107us/step - loss: 0.9025 - acc: 0.8378
Epoch 80/700
222/222 [==============================] - 0s 112us/step - loss: 0.9171 - acc: 0.8198
Epoch 81/700
222/222 [==============================] - 0s 112us/step - loss: 0.8509 - acc: 0.8559
Epoch 82/700
222/222 [==============================] - 0s 105us/step - loss: 0.8419 - acc: 0.8423
Epoch 83/700
222/222 [==============================] - 0s 106us/step - loss: 0.9652 - acc: 0.8018
Epoch 84/700
222/222 [==============================] - 0s 112us/step - loss: 0.7596 - acc: 0.8604
Epoch 85/700
222/222 [=

222/222 [==============================] - 0s 117us/step - loss: 0.4706 - acc: 0.9144
Epoch 158/700
222/222 [==============================] - 0s 105us/step - loss: 0.4094 - acc: 0.8874
Epoch 159/700
222/222 [==============================] - 0s 112us/step - loss: 0.5418 - acc: 0.8829
Epoch 160/700
222/222 [==============================] - 0s 109us/step - loss: 0.3607 - acc: 0.9459
Epoch 161/700
222/222 [==============================] - 0s 111us/step - loss: 0.3784 - acc: 0.9144
Epoch 162/700
222/222 [==============================] - 0s 108us/step - loss: 0.3215 - acc: 0.9279
Epoch 163/700
222/222 [==============================] - 0s 108us/step - loss: 0.3448 - acc: 0.9369
Epoch 164/700
222/222 [==============================] - 0s 116us/step - loss: 0.4153 - acc: 0.9234
Epoch 165/700
222/222 [==============================] - 0s 106us/step - loss: 0.3921 - acc: 0.9054
Epoch 166/700
222/222 [==============================] - 0s 111us/step - loss: 0.3344 - acc: 0.9324
Epoch 167/700


222/222 [==============================] - 0s 122us/step - loss: 0.3241 - acc: 0.9459
Epoch 240/700
222/222 [==============================] - 0s 116us/step - loss: 0.3702 - acc: 0.9279
Epoch 241/700
222/222 [==============================] - 0s 115us/step - loss: 0.2930 - acc: 0.9414
Epoch 242/700
222/222 [==============================] - 0s 115us/step - loss: 0.3700 - acc: 0.9414
Epoch 243/700
222/222 [==============================] - 0s 113us/step - loss: 0.2970 - acc: 0.9505
Epoch 244/700
222/222 [==============================] - 0s 113us/step - loss: 0.2721 - acc: 0.9279
Epoch 245/700
222/222 [==============================] - 0s 109us/step - loss: 0.3902 - acc: 0.9459
Epoch 246/700
222/222 [==============================] - 0s 112us/step - loss: 0.3638 - acc: 0.9324
Epoch 247/700
222/222 [==============================] - 0s 110us/step - loss: 0.2697 - acc: 0.9459
Epoch 248/700
222/222 [==============================] - 0s 109us/step - loss: 0.3549 - acc: 0.9279
Epoch 249/700


...

222/222 [==============================] - 0s 148us/step - loss: 0.1406 - acc: 0.9730
Epoch 404/700
222/222 [==============================] - 0s 137us/step - loss: 0.1376 - acc: 0.9640
Epoch 405/700
222/222 [==============================] - 0s 117us/step - loss: 0.1963 - acc: 0.9595
Epoch 406/700
222/222 [==============================] - 0s 111us/step - loss: 0.1972 - acc: 0.9595
Epoch 407/700
222/222 [==============================] - 0s 105us/step - loss: 0.1473 - acc: 0.9730
Epoch 408/700
222/222 [==============================] - 0s 107us/step - loss: 0.1702 - acc: 0.9640
Epoch 409/700
222/222 [==============================] - 0s 106us/step - loss: 0.1885 - acc: 0.9595
Epoch 410/700
222/222 [==============================] - 0s 111us/step - loss: 0.1325 - acc: 0.9775
Epoch 411/700
222/222 [==============================] - 0s 108us/step - loss: 0.1229 - acc: 0.9730
Epoch 412/700
222/222 [==============================] - 0s 109us/step - loss: 0.1753 - acc: 0.9640
Epoch 413/700


222/222 [==============================] - 0s 105us/step - loss: 0.1009 - acc: 0.9775
Epoch 486/700
222/222 [==============================] - 0s 103us/step - loss: 0.1078 - acc: 0.9730
Epoch 487/700
222/222 [==============================] - 0s 106us/step - loss: 0.1548 - acc: 0.9595
Epoch 488/700
222/222 [==============================] - 0s 106us/step - loss: 0.0925 - acc: 0.9820
Epoch 489/700
222/222 [==============================] - 0s 107us/step - loss: 0.1236 - acc: 0.9730
Epoch 490/700
222/222 [==============================] - 0s 104us/step - loss: 0.1668 - acc: 0.9595
Epoch 491/700
222/222 [==============================] - 0s 104us/step - loss: 0.1129 - acc: 0.9730
Epoch 492/700
222/222 [==============================] - 0s 104us/step - loss: 0.0906 - acc: 0.9775
Epoch 493/700
222/222 [==============================] - 0s 102us/step - loss: 0.1762 - acc: 0.9550
Epoch 494/700
222/222 [==============================] - 0s 102us/step - loss: 0.0674 - acc: 0.9910
Epoch 495/700


222/222 [==============================] - 0s 111us/step - loss: 0.1169 - acc: 0.9685
Epoch 568/700
222/222 [==============================] - 0s 106us/step - loss: 0.0519 - acc: 0.9910
Epoch 569/700
222/222 [==============================] - 0s 114us/step - loss: 0.1300 - acc: 0.9685
Epoch 570/700
222/222 [==============================] - 0s 111us/step - loss: 0.0988 - acc: 0.9820
Epoch 571/700
222/222 [==============================] - 0s 108us/step - loss: 0.1236 - acc: 0.9685
Epoch 572/700
222/222 [==============================] - 0s 112us/step - loss: 0.1192 - acc: 0.9685
Epoch 573/700
222/222 [==============================] - 0s 112us/step - loss: 0.1053 - acc: 0.9685
Epoch 574/700
222/222 [==============================] - 0s 108us/step - loss: 0.0807 - acc: 0.9775
Epoch 575/700
222/222 [==============================] - 0s 113us/step - loss: 0.0906 - acc: 0.9730
Epoch 576/700
222/222 [==============================] - 0s 109us/step - loss: 0.0997 - acc: 0.9730
Epoch 577/700


222/222 [==============================] - 0s 107us/step - loss: 0.0783 - acc: 0.9775
Epoch 650/700
222/222 [==============================] - 0s 107us/step - loss: 0.0978 - acc: 0.9820
Epoch 651/700
222/222 [==============================] - 0s 106us/step - loss: 0.0227 - acc: 1.0000
Epoch 652/700
222/222 [==============================] - 0s 109us/step - loss: 0.1658 - acc: 0.9505
Epoch 653/700
222/222 [==============================] - 0s 107us/step - loss: 0.1383 - acc: 0.9640
Epoch 654/700
222/222 [==============================] - 0s 106us/step - loss: 0.0649 - acc: 0.9820
Epoch 655/700
222/222 [==============================] - 0s 111us/step - loss: 0.0753 - acc: 0.9865
Epoch 656/700
222/222 [==============================] - 0s 108us/step - loss: 0.0664 - acc: 0.9820
Epoch 657/700
222/222 [==============================] - 0s 109us/step - loss: 0.0916 - acc: 0.9685
Epoch 658/700
222/222 [==============================] - 0s 109us/step - loss: 0.1630 - acc: 0.9505
Epoch 659/700


INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
Processed actions: 222it [00:00, 1588.17it/s, # examples=222]
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to '/Users/saching12/Desktop/python codes/student_assistant_chatbot/models/dialogue'


# Talk to your Bot

In [84]:
#Starting the Bot

from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

INFO:tensorflow:Restoring parameters from /Users/saching12/Desktop/python codes/student_assistant_chatbot/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt


INFO:tensorflow:Restoring parameters from /Users/saching12/Desktop/python codes/student_assistant_chatbot/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt


In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_text(a)
    for response in responses:
        print(response["text"])
        

Your bot is ready to talk! Type your messages here or send 'stop'
is there any event tomorrow?


ERROR:rasa_core.processor:Encountered an exception while running action 'action_retrieve_event'. Bot will continue, but the actions events are lost. Make sure to fix the exception in your custom code.


Did that help you?
